In [ ]:
import os
import multiprocessing
from coulombexp_sim_2021_routines import*

In [ ]:
os.chdir(r'C:\Users\chem-univ4894\OneDrive - Nexus365\Documents\Project\cei_simulations\ethene')
input_file = 'ethene_channels.txt'
shots = 20000

# Create the object 'experiment', an instance of 'Experiment' that holds the information about the system to simulate.
experiment = read_instructions(input_file)
experiment.sigma = 0
experiment.epsilon = 1

# Event rate is controlled by an augmented Poisson distribution, where the centre of the Poisson distribution is drawn from a 
# normal distribution centered around 1. Bounds are zero and an arbitrary upper limit (negative event rate is unphysical).
mu = 10
lower, upper = 0, 1000
Gamma_dist = experiment.normaldist(experiment.sigma, upper, lower, mu)

In [ ]:
def cycle_over_shots(shots, expmnt=experiment):
    """
    Runs the simulation defined in the above functions over a number of
    experimental cycles (shots). Each time samples the required probability
    distributions to get the number of explosions to simulate. Splits these
    into the possible fragmentation pathways with a multinomial distribution.
    Collects all ion properties at tf into DataFrame data.
    Parameters
    -----------
    - shots:array of integers corresponding to the laser shot numbers to simulate.
    - expmnt: the Experiment object holding the experimental parameters and geometry etc.
      Has to be a default argument as pool.map doesn't handle multiple arguments well.
    Returns
    --------
    - data: DataFrame of ion final positions, velocities, masses, charges and shot numbers for post-processing
      of results. Only contains 'detected' ions (not thrown out by explode2).

    """
    sigma = expmnt.sigma
    if sigma != 0:
        Gamma_dist = expmnt.Gamma_dist
    else:
        pass
#     xs, ys, zs, ts, ms, qs, vxs, vys, vzs, ls, gammas = [], [], [], [], [], [], [], [], [], [], []
    xs, ys, zs, ts, ms, qs, vxs, vys, vzs, ls = [], [], [], [], [], [], [], [], [], []
    for shot in shots:
        print(shot)
        rng = default_rng()
        if sigma == 0:
            v = expmnt.v0
        else:
            gamma = float(Gamma_dist.rvs(1))
#             print(gamma)
            v = gamma*expmnt.v0
        n = rng.poisson(v)
        #n = v
        molecules_per_channel = rng.multinomial(n, expmnt.channel_probs)
        molecules_in_channel = {i: None for i in range(0, expmnt.no_channels)}
        for i in range(0, expmnt.no_channels):
            numbertocreate = molecules_per_channel[i]
            molecules = []
            for x in range(0, numbertocreate):
                j = expmnt.createMoleculeObj()
                molecules.append(j)
            for m in molecules:
                m.vib()
            molecules_in_channel[i] = molecules
        for i in range(0, expmnt.no_channels):
            m = molecules_in_channel[i]
            for ml in m:
                ml.createFragments(i, expmnt)
        allmolecules = []
        for m in molecules_in_channel.values():
            allmolecules.extend(m)
        (x, y, z, t, m, q, vx , vy, vz) = explode2(allmolecules, expmnt.epsilon)
        xs.extend(x)
        ys.extend(y)
        zs.extend(z)
        ts.extend(t)
        ms.extend(m)
        qs.extend(q)
        vxs.extend(vx)
        vys.extend(vy)
        vzs.extend(vz)
        shot_instance = np.full_like(x, shot)
        ls.extend(shot_instance)
#         gamma_instance = np.full_like(x, gamma)
#         gammas.extend(gamma_instance)
    s = list_of_lists_to_list_of_series([xs, ys, zs, ts, ms, qs, vxs, vys, vzs, ls])
#     s = list_of_lists_to_list_of_series([xs, ys, zs, ts, ms, qs, vxs, vys, vzs, ls, gammas])
#     print(s)
    data = pd.concat(s, axis = 1)
    data.columns =  ['x', 'y', 'z', 't', 'm', 'q', 'vx' , 'vy', 'vz', 'shot']
#     data.columns =  ['x', 'y', 'z', 't', 'm', 'q', 'vx' , 'vy', 'vz', 'shot', 'gamma']
    return data

In [ ]:
laser_shots = np.arange(1, shots+1)
df = cycle_over_shots(laser_shots)
print(df)
outname = '{}_{}_shots_sigma{}_v0{}_eps{}_higheventrate10.csv'.format(str(experiment.name), shots, experiment.sigma, experiment.v0, experiment.epsilon)
df.to_csv(outname, header=True, index=False)
print('Saved Data to {}'.format(outname))

# laser_shots = np.arange(1, shots+1)
# # Split the shots so a different CPU handles each.
# num_processes = multiprocessing.cpu_count()
# divided_shots = np.array_split(laser_shots, num_processes)
# # print(divided_shots)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df.x, df.y, df.z)
plt.show()

In [ ]:
# Convert output into PImMS format:

coulomb_explosion_output = outname

df = pd.read_csv(coulomb_explosion_output, header='infer')
print(df)
df['radial_extent'] = np.sqrt((df.x**2) + (df.y**2) + (df.z**2))
maxrad = df.radial_extent.max()
pixs = 324
x_bins = np.linspace(-maxrad, maxrad, pixs)  # create the bins for x and y data
y_bins = x_bins
# tbins = np.linspace(0, 1.024e-4, num=4096) # For time-of-flights
tbins = np.linspace(0, 1000, num=1001) # If using m/z only
print(tbins)

def list_of_lists_to_list_of_series(ls):  # useful space saver
    s = []
    for l in ls:
        s0 = pd.Series(l)
        s.append(s0)
    return s

def get_pixel(dataframe, xbins = x_bins, ybins = y_bins, timebins = tbins):
    xpix = np.digitize(dataframe.x, xbins)
    ypix = np.digitize(dataframe.y, ybins)
#     times_binned = np.digitize(dataframe.t, tbins) # For Tof
#     times_binned = list( map(int, times_binned))
    times_binned = np.digitize(dataframe.m/dataframe.q, tbins) # For m/z
    times_binned = list( map(int, times_binned))
    times_binned = [x - 1 for x in times_binned]
    shots = list(dataframe.shot)
    shots = list( map(int, shots) )
#     gammas = list(dataframe.gamma)
    ls = [xpix, ypix, times_binned, shots]
#     ls = [xpix, ypix, times_binned, shots, gammas]
    s = list_of_lists_to_list_of_series(ls)
    data = pd.concat(s, axis = 1)
    return data

pixel_output = get_pixel(df)

print(pixel_output)

outname2 = outname[:-4]+'_pixelated.csv'
pixel_output.to_csv(outname2, header=True, index=False)

In [ ]:
# 3-1. Create and plot PImMS image. 

# User input required in sections denoted by "#!".

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import pandas as pd
import time
import os

#! Image time window (Ti to Tf) and size (Pixels: PImMS1 = 72, PImMS2 = 324):
Ti = 13  #H+ 
Tf = 15 #H+ 
Pixels = 324


#! Reads data to create an X,Y,T,Shot dataframe:
df = pd.read_csv(outname2, names=['x','y','t','shot'], header=0)
print(df)

# #! Read XYT data, create XYI file for a specific T range, plot image:
df = df[(df['t'] > Ti) & (df['t'] < Tf)] # Filters data by time
df = df.groupby(['x','y']).size().reset_index(name='intensity')
print(df) # time-filtered data-frame
image = np.zeros((Pixels,Pixels))
results = np.array(df)
results = results.astype(int)
for a, b, c in results:
    image[b,a] = c # [b,a] plots x horizontally starting from zero at the left, and y vertically starting from zero at the top.
# print(image)
# print(image.shape)
# print(df)


# Plots PImMS image (modify to suit taste):
fig2, ax = plt.subplots(figsize=(6, 6))
maximage = image.max()
picture = ax.imshow(image, interpolation='nearest', cmap='inferno', vmin=0, vmax=0.65*maximage)
ax.set_title('CH$^{+}$ - Simulated CEI data (20000 shots)', loc='center', fontsize=16, y=1.03)
ax.set_xlim(0,324)
ax.set_ylim(324,0) # Inverted to keep x,y directions the same as above
ax.set_xlabel('$x$', fontsize=12)
ax.set_ylabel('$y$', fontsize=12)
ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
cax.tick_params(labelsize=14)
plt.colorbar(picture, cax=cax, ticks=[0, 50], orientation='vertical').set_label(label='Intensity (arb. units) \n', size=12, labelpad=0, rotation=270)
fig2.savefig('ethene_monocationfrag_20000shots_pimms.png')




    